In [44]:
#pip install faiss-cpu sentence-transformers openai

In [45]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

In [46]:
recipe_faiss_file = 'recipes_dataset\parsed_recipes.txt'
conversations_faiss_file = 'conversations_dataset\parsed_conversations.txt'

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [48]:
recipes_index = faiss.read_index("recipes.faiss")
conversations_index = faiss.read_index("conversations.faiss")

In [49]:
def split_text(source_file, delimiter = '==CHUNK END=='):
    with open(source_file, "r", encoding="utf-8") as f:
        content = f.read()
    return content.split(delimiter)

In [50]:
recipes_chunks = split_text('recipes_dataset\parsed_recipes.txt')
conversations_chunks = split_text('conversations_dataset\parsed_conversations.txt')

In [ ]:
question = "I'm sad, what should i eat, can you give me a recipe?"

q_vector = model.encode([question])

D1, I1 = recipes_index.search(np.array(q_vector), k=1)
D2, I2 = conversations_index.search(np.array(q_vector), k=1)

recipes_chunks = [recipes_chunks[i] for i in I1[0]]
conversations_chunks = [conversations_chunks[i] for i in I2[0]]


In [52]:
for i, chunk in enumerate(recipes_chunks):
    print(f"Chunk {i+1}: {chunk}")

Chunk 1: 
Title: Sour Cream 'N Dill Chicken
Ingredients: ["8 to 10 chicken pieces, skinned", "pepper", "1 can condensed cream of mushroom soup (undiluted)", "1 envelope dry onion soup mix", "1 c. (8 oz.) sour cream", "1 Tbsp. lemon juice", "1 Tbsp. fresh dill, chopped or 1 tsp. dill weed", "1 (4 oz.) can sliced mushrooms, drained", "paprika", "cooked wide egg noodles (optional)"]
Directions: ["Dip pieces of chicken in melted margarine, then roll pieces in crushed Rice Krispies.", "Season to taste.", "Place in pan lined with aluminum foil.", "Bake for 1 hour at 350\u00b0."]



In [53]:
context = "\n\n".join(recipes_chunks) + "\n\n".join(conversations_chunks)

prompt = f"""
Context:
{context}

Question: {question}
Answer:
"""

In [ ]:
from openai import OpenAI

client = OpenAI(base_url="http://localhost:1234/v1", api_key="sk-no-key-needed")


response = client.chat.completions.create(
    model="mistral",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.7,
)

print(response.choices[0].message.content)


 Title: Creamy Tomato Pasta
Ingredients: ["1 lb. (450 g) spaghetti", "2 Tbsp. olive oil", "3 garlic cloves, minced", "1 (28 oz.) can crushed tomatoes", "1 tsp. sugar", "1 tsp. dried basil", "1/2 tsp. salt", "1/4 tsp. black pepper", "2 c. shredded mozzarella cheese", "1/2 cup grated Parmesan cheese"]
Directions: ["Cook spaghetti according to package directions.", "Heat olive oil in a large skillet over medium heat.", "Add garlic and cook until fragrant, about 1 minute.", "Stir in crushed tomatoes, sugar, basil, salt, and pepper.", "Simmer for 10 minutes.", "Drain spaghetti and add to the skillet.", "Stir in mozzarella cheese and Parmesan cheese until melted.", "Serve hot."]

I'm sorry to hear that you're feeling sad. Comfort food can sometimes help lift our mood, and this Creamy Tomato Pasta recipe might be a great choice for you. It's easy to make, and the combination of tomatoes, basil, mozzarella, and Parmesan cheese should provide a delicious, comforting meal. Hope it helps!
